<a href="https://colab.research.google.com/github/bimadewantoro/indonesian-news-summarization/blob/main/Abstractive_Summarization_Indonesia_Datasheet_Endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Module

In [3]:
!pip install torch
!pip install transformers
!pip install sentencepiece
!pip install colabcode
!pip install flask
!pip install flask-ngrok
!pip install jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 40.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 90.4 MB/s eta 0:00:00
     

Import Module

In [4]:
import torch
import nltk
from nltk.corpus import stopwords
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from colabcode import ColabCode
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

Define Flask App

In [28]:
app = Flask(__name__)
run_with_ngrok(app)
!ngrok authtoken 2KwoaOX9u62fDd4tJisKC6QdsPu_3Gfo2Lw4s3YjU5Dc8PbQj

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Load model

In [29]:
model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased")
tokenizer = T5Tokenizer.from_pretrained("cahya/t5-base-indonesian-summarization-cased")

In [30]:
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
def preprocess_text(text):
    text = "summarize: " + text
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text

In [32]:
def remove_duplicates(text):
    sentences = text.split('. ')
    unique_sentences = []
    for sentence in sentences:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return '. '.join(unique_sentences)

Define function to postprocess tokens

In [33]:
def postprocess_text(text):
    # Remove any leading or trailing spaces and <s>, </s> tags
    text = text.strip().replace("<s>", "").replace("</s>", "")
    return text

Define function to Summary

In [34]:
def predict(text):
    # Preprocess text
    text = preprocess_text(text)
    # Encode text
    text = tokenizer.encode(text, return_tensors="pt", truncation=True)
    # Generate summary
    summary_ids = model.generate(text, 
            min_length=20,
            max_length=80,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)
    # Decode summary
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    # Postprocess summary
    output = postprocess_text(output)
    return output

Define route

In [35]:
# Define route
@app.route('/', methods=['GET'])
def home():
    return '''<h1>Summarize API</h1>
    <p>A prototype API for summarizing text.</p>'''

@app.route('/summary', methods=['POST'])
def summary_route():
    # Get text format
    content_type = request.headers['Content-Type']

    # Extract text based on format
    if content_type == 'application/json':
        text = request.json['text']
    elif content_type == 'application/text':
        text = request.data.decode('utf-8')
    elif content_type == 'application/x-www-form-urlencoded':
        text = request.form['text']
    else:
        return jsonify({'error': 'Invalid content type'}), 400

    # Predict summary
    summary = predict(text)
    # Return summary
    return jsonify({'summary': summary})

Run App

In [ ]:
if __name__ == '__main__':
    ColabCode(port=10000, code=False)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://077a-35-230-181-214.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [05/Feb/2023 17:47:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2023 17:47:24] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2023 17:47:28] "GET /summary HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [05/Feb/2023 17:47:29] "GET /favicon.ico HTTP/1.1" 404 -
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:werkzeug:127.0.0.1 - - [05/Feb/2023 17:47:59] "POST /summary HTTP/1.1" 200 -
